# 利用 Google API 抓取資料

## 1.取得上市股票代號

### [上市上櫃下市股票列表SQLite資料庫](https://github.com/ouvek-kostiva/live4program/blob/master/Ep4/list.db)

In [2]:
def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList

codeList = getListedCode("twse",4) #取得上市股票代碼 twse, 長度4碼
print(codeList[0:5])

取得股票代號總數: 913
['1101', '1102', '1103', '1104', '1108']


## 2.查詢 Google API

#### q 股票代碼
#### x 交易所 [代碼列表](https://www.google.com/googlefinance/disclaimer/)
#### i 時間間隔(秒數)
#### p 幾個時間單位
#### f=d,c,h,l,o,v 取得那些欄位(日期, 收盤, 最高, 最低, 開盤, 交易量)

In [5]:
def getGoogleData(stockCode, exchange="TPE", interval="86400", duration="10Y"):
    f = "d,c,h,l,o,v" # Fields : Columns > d,c,h,l,o,v = Date + ?, Close, High, Low, Open, Volume
    import requests
    import pickle
    r = requests.get('https://www.google.com/finance/getprices?q={}&x={}&i={}&p={}&f={}'.format(stockCode,exchange,interval,duration,f))
    output = open('Pickles/{}_{}.pkl'.format(stockCode,duration), 'wb')
    print("write:","{}_{}.pkl".format(stockCode,duration))
    lines = r.text.split('\n')
    pickle.dump(lines, output) #寫入 pickle
    pklname = '{}_{}.pkl'.format(stockCode,duration)
    return pklname

code = '1101'
pklname = getGoogleData(code, exchange="TPE", interval="86400", duration="3d")
print(pklname)

write: 1101_3d.pkl
1101_3d.pkl


## 3.讀取 Pickle 資料

#### Python 中可將任意物件先以 Pickle 存好後再讀取使用

In [7]:
def readPickledData(fileName):
    import os
    import pickle
    if os.path.isfile('Pickles/{}.pkl'.format(fileName)):
        pkl_file = open('Pickles/{}.pkl'.format(fileName), 'rb')
        lines = pickle.load(pkl_file)
        print("Pickled File ", fileName, " Loaded")
        return lines
    else:
        print("Filename should be like: code_duration, ex:1101_10Y")
        
fileName = '1101_3d'
lines = readPickledData(fileName)

for item in lines:
    print(item)

Pickled File  1101_3d  Loaded
EXCHANGE%3DTPE
MARKET_OPEN_MINUTE=540
MARKET_CLOSE_MINUTE=810
INTERVAL=86400
COLUMNS=DATE,CLOSE,HIGH,LOW,OPEN,VOLUME
DATA=
TIMEZONE_OFFSET=480
a1496295000,34.35,34.6,34.35,34.4,4060000
1,34.6,34.7,34.4,34.4,3264000



## 4.建立新資料庫

#### 欄位: indexColumn 主鍵, code 股票代號, date 日期, close 收盤價, low 最低價, open 開盤價, volume 交易量
#### [其他資料庫教學](http://tw.gitbook.net/sqlite/sqlite_create_table.html)

In [8]:
def createDatabase(dbName,tableName):
    import os
    if os.path.isfile(dbName):
        return "Database name already exists: ", dbName
    else:    
        import sqlite3
        sqlite_file = dbName
        conn = sqlite3.connect(sqlite_file)
        c = conn.cursor()
        c.execute('CREATE TABLE {tn} (indexColumn INTEGER PRIMARY KEY, code TEXT NOT NULL, date TEXT NOT NULL, close REAL, high REAL, low REAL, open REAL, volume REAL)'.format(tn=tableName)) 
        conn.commit()
        conn.close()
        return "Database",dbName," Successfully Created!"
    
dbName = "PriceVolData.db"
tableName = "twse"
createDatabase(dbName,tableName)

('Database', 'PriceVolData.db', ' Successfully Created!')

### 如同名資料庫或資料表存在

In [9]:
dbName = "PriceVolData.db"
tableName = "twse"
createDatabase(dbName,tableName)

('Database name already exists: ', 'PriceVolData.db')

## 5.整理將從 API 取得的資料

In [11]:
def toDataList(fileName):
    import datetime as dt
    lines = readPickledData(fileName)
    print("初始:",lines) #非教學時請刪除此輸出行
    del lines[0:7]
    print("刪除前面資料:",lines) #非教學時請刪除此輸出行
    dataList = []
    dtDate = 0;
    for ind,lin in enumerate(lines[:-1]):
        spl = lin.split(",")
        print("分開欄位:",spl) #非教學時請刪除此輸出行
        if spl[0][0] == 'a':
            print(spl[0][1:])
            actDate = dt.datetime.fromtimestamp(int(spl[0][1:])).strftime('%Y-%m-%d %H:%M:%S')
            print("Unix時間:",actDate) #非教學時請刪除此輸出行
            dtDate = dt.datetime.strptime(actDate, '%Y-%m-%d %H:%M:%S')
        else:
            add = int(spl[0])
            print("Unix時間後增加天數:",add) #非教學時請刪除此輸出行
            newDate = dtDate + dt.timedelta(days=add)
            print("後續時間:",newDate) #非教學時請刪除此輸出行
            fDate = newDate.strftime('%Y-%m-%d %H:%M:%S') #fDate 日期字串
            close = spl[1]
            high = spl[2]
            low = spl[3]
            ope = spl[4]
            vol = spl[5]
            print("日期, 收盤, 最高, 最低, 開盤, 交易量",spl[1],spl[2],spl[3],spl[4],spl[5]) #非教學時請刪除此輸出行
            dataList.append([fDate, close, high, low, ope, vol])
            print("dataList:",dataList) #非教學時請刪除此輸出行
    return dataList

fileName = '1101_3d'
dataList = toDataList("{}".format(fileName))
print("整理結果:",dataList)

Pickled File  1101_3d  Loaded
初始: ['EXCHANGE%3DTPE', 'MARKET_OPEN_MINUTE=540', 'MARKET_CLOSE_MINUTE=810', 'INTERVAL=86400', 'COLUMNS=DATE,CLOSE,HIGH,LOW,OPEN,VOLUME', 'DATA=', 'TIMEZONE_OFFSET=480', 'a1496295000,34.35,34.6,34.35,34.4,4060000', '1,34.6,34.7,34.4,34.4,3264000', '']
刪除前面資料: ['a1496295000,34.35,34.6,34.35,34.4,4060000', '1,34.6,34.7,34.4,34.4,3264000', '']
分開欄位: ['a1496295000', '34.35', '34.6', '34.35', '34.4', '4060000']
1496295000
Unix時間: 2017-06-01 13:30:00
分開欄位: ['1', '34.6', '34.7', '34.4', '34.4', '3264000']
Unix時間後增加天數: 1
後續時間: 2017-06-02 13:30:00
日期, 收盤, 最高, 最低, 開盤, 交易量 34.6 34.7 34.4 34.4 3264000
dataList: [['2017-06-02 13:30:00', '34.6', '34.7', '34.4', '34.4', '3264000']]
整理結果: [['2017-06-02 13:30:00', '34.6', '34.7', '34.4', '34.4', '3264000']]


## 5.資料輸入資料庫

In [13]:
def insertData(dbName, tableName, dataList, stockCode):
    import sqlite3
    conn = sqlite3.connect(dbName)
    c = conn.cursor()
    count = 0
    for date, close, high, low, ope, vol in dataList:
        conn.execute("INSERT INTO twse (code, date, close, high, low, open, volume) VALUES (?,?,?,?,?,?,?)",(stockCode, date, close, high, low, ope, vol))
        count = count + 1
    conn.commit()
    conn.close()
    return count

dbName = "PriceVolData.db"
tableName = "twse"
code = '1101'
for item in dataList:
    print(item)
count = insertData(dbName, tableName, dataList, code)
print("輸入筆數:",count)

['2017-06-02 13:30:00', '34.6', '34.7', '34.4', '34.4', '3264000']
輸入筆數: 1


## 6.資料從資料庫讀取

In [16]:
import sqlite3
dbName = "PriceVolData.db"
tableName = "twse"
conn = sqlite3.connect(dbName)
c = conn.cursor()

out = conn.execute("SELECT * FROM twse")

dalist = []
for i in out:
    dalist.append([i])
    
conn.close()

for item in dalist:
    print(item)

[(1, '1101', '2017-06-02 13:30:00', 34.6, 34.7, 34.4, 34.4, 3264000.0)]
[(2, '1101', '2017-06-02 13:30:00', 34.6, 34.7, 34.4, 34.4, 3264000.0)]
